- Download dataset from kaggle (kaggle dataset)
- Download the validate image set
- Show some images
- Use the default Yolov8 model to detect object in kaggle dataset
- Train 1st model
- Use 1st model to detect object in kaggle dataset -> good
- Use 1st model to detect object validate image set -> bad
- Use team model to detect object validate image set -> neutral , have a better result
-> The 1st model have been overfitted




In [1]:

!pip install -q roboflow
!pip install -q unzip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:

!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -e .


Cloning into 'ultralytics'...
remote: Enumerating objects: 41527, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 41527 (delta 132), reused 163 (delta 55), pack-reused 41262 (from 1)
Receiving objects: 100% (41527/41527), 31.30 MiB | 20.85 MiB/s, done.
Resolving deltas: 100% (30767/30767), done.
/content/ultralytics
Obtaining file:///content/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.95-0.editable-py3-none-any.whl size=23033 sha256=4cbe674d88a5bb7f188f1fb02f086677c0de0456625991c969df89f87ad60108
  Stored in directory: /tmp/pip-ephem-wheel-cache-vmommd9l/wheels/9a/cd/d5/95912172899f8ec640166ff6eef49156b1b00d6b2ade4a3cb1
Succe

In [3]:
import os
import random
import pandas as pd
from PIL import Image
import cv2
from ultralytics import YOLO
from IPython.display import Video
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')
import pathlib
import glob
from tqdm.notebook import trange, tqdm
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Configure the visual appearance of Seaborn plots
sns.set(rc={'axes.facecolor': '#eae8fa'}, style='darkgrid')

In [5]:
# Step 1: Upload kaggle.json
from google.colab import files
files.upload()  # This will prompt you to upload the kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mnhthinguyn","key":"8e929ac7632d9908a476eb9c12e2d0e4"}'}

In [ ]:
# Step 2: Move kaggle.json to the correct location
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Step 3: Set permissions
!chmod 600 ~/.kaggle/kaggle.json
# Install Kaggle package
!pip install kaggle

In [7]:
!mkdir -p /content/specific-folder
!kaggle datasets download -d mnhthinguyn/traffic-sign-50-60-70-self-trainning -p /content/specific-folder

Dataset URL: https://www.kaggle.com/datasets/mnhthinguyn/traffic-sign-50-60-70-self-trainning
License(s): unknown
 25% 5.00M/19.7M [00:00<00:00, 17.8MB/s]
100% 19.7M/19.7M [00:00<00:00, 59.1MB/s]


In [ ]:
!unzip /content/specific-folder/traffic-sign-50-60-70-self-trainning.zip -d /content/dataset

Show Some Images From TrainSet

In [ ]:
Image_dir = '/content/dataset/Self_trainning_sign/Self_trainning/train/images/50kmh'

num_samples = 9
image_files = os.listdir(Image_dir)

# Randomly select num_samples images
rand_images = random.sample(image_files, num_samples)

fig, axes = plt.subplots(3, 3, figsize=(11, 11))

for i in range(num_samples):
    image = rand_images[i]
    ax = axes[i // 3, i % 3]
    ax.imshow(plt.imread(os.path.join(Image_dir, image)))
    ax.set_title(f'Image {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

Evaluate the image size on the image folder

Download the validate

In [ ]:
%cd /content
!wget http://14.225.204.171:9998/validate_data_2.zip
!wget http://14.225.204.171:9998/best_0827.pt
!wget http://14.225.204.171:9998/best.pt
!unzip /content/validate_data_2.zip

Check the validate item

In [ ]:
Image_dir = '/content/validate/images'

num_samples = 9
image_files = os.listdir(Image_dir)

# Randomly select num_samples images
rand_images = random.sample(image_files, num_samples)

fig, axes = plt.subplots(3, 3, figsize=(11, 11))

for i in range(num_samples):
    image = rand_images[i]
    ax = axes[i // 3, i % 3]
    ax.imshow(plt.imread(os.path.join(Image_dir, image)))
    ax.set_title(f'Image {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

Summary the model result before trainning

In [ ]:
from ultralytics import YOLO
import os
import cv2
from google.colab.patches import cv2_imshow  # Use this for displaying images in Google Colab

# Load the pretrained YOLOv8n model
model = YOLO("yolov8n.pt")

# Define the folder containing the images
folder_path = "/content/dataset/Self_trainning_sign/Self_trainning/train/images/50kmh"

# Initialize counters
total_images = 0
images_with_detections = 0

# Loop through each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Ensure file is an image
        image_path = os.path.join(folder_path, filename)
        total_images += 1

        # Use the model to detect objects in the image, suppress verbose output
        result_predict = model.predict(source=image_path, imgsz=(640), verbose=False)

        # Check if there are any detections
        if len(result_predict[0].boxes) > 0:  # Check if there are any bounding boxes
            images_with_detections += 1
            print(f"Detections found in image: {filename}")

            # Display the detected image
            detected_img = result_predict[0].plot()  # Plot the detections on the image
            cv2_imshow(detected_img)  # Display the image in Google Colab

# Calculate the percentage of images with detections
if total_images > 0:
    detection_percentage = (images_with_detections / total_images) * 100
else:
    detection_percentage = 0

# Print the overall report
print(f"Total images processed: {total_images}")
print(f"Images with detections: {images_with_detections}")
print(f"Percentage of images with detections: {detection_percentage:.2f}%")


In [ ]:
!yolo task=detect mode=train model=/content/dataset/Self_trainning_sign/Self_trainning/4th_periord_best.pt data=/content/dataset/Self_trainning_sign/Self_trainning/data1.yaml epochs=20 imgsz=640 batch=16

import os
import glob
from google.colab import files

# Define the base path where YOLO saves the training results
base_path = "/content/ultralytics/runs/detect/"

# Get a list of all training folders
train_folders = glob.glob(os.path.join(base_path, 'train*'))

# Find the most recently modified training folder
latest_train_folder = max(train_folders, key=os.path.getmtime)

# Construct the path to the best model
model_path = os.path.join(latest_train_folder, "weights", "best.pt")

# Check if the model file exists and download it
if os.path.exists(model_path):
    print(f"Training completed. Model found in {latest_train_folder}. Preparing to download...")
    files.download(model_path)
else:
    print("Model not found. Please check the training process and paths.")

## Validate the model after training with the the valid dataset extract from the trained dataset

# Validate the model after training with the the valid dataset extract from the trained dataset with images result suggesstion

In [ ]:
from ultralytics import YOLO
import os
import sys
import cv2
from google.colab.patches import cv2_imshow  # Use this for displaying images in Google Colab
from contextlib import redirect_stdout
import logging

# Suppress logging messages
logging.getLogger().setLevel(logging.ERROR)

# Load the pretrained YOLOv8n model
model = YOLO("/content/best.pt")  # Load your custom-trained model

# Define the folder containing the images
folder_path = "/content/dataset/Self_trainning_sign/Self_trainning/valid/images/50kmh"

# Initialize counters
total_images = 0
images_with_detections = 0

# Suppress standard output to ignore detailed logs
with open(os.devnull, 'w') as fnull:
    with redirect_stdout(fnull):
        # Loop through each image in the folder
        for filename in os.listdir(folder_path):
            if filename.endswith(('.png', '.jpg', '.jpeg')):  # Ensure file is an image
                image_path = os.path.join(folder_path, filename)
                total_images += 1

                # Use the model to detect objects in the image, verbose=False to suppress logging
                result_predict = model.predict(source=image_path, imgsz=(640), verbose=False)

                # Check if there are any detections
                if len(result_predict[0].boxes) > 0:  # Check if there are any bounding boxes
                    images_with_detections += 1

                    # Display the image with detections
                    detected_img = result_predict[0].plot()  # Plot the detections on the image
                    cv2_imshow(detected_img)  # Display the detected image in Colab

# Calculate the percentage of images with detections
if total_images > 0:
    detection_percentage = (images_with_detections / total_images) * 100
else:
    detection_percentage = 0

# Print the overall report
print(f"Total images processed: {total_images}")
print(f"Images with detections: {images_with_detections}")
print(f"Percentage of images with detections: {detection_percentage:.2f}%")


In [ ]:
%cd /content
!wget http://14.225.204.171:9998/validate_data_2.zip
!unzip /content/validate_data_2.zip

In [ ]:
Image_dir = '/content/validate/images'

num_samples = 9
image_files = os.listdir(Image_dir)

# Randomly select num_samples images
rand_images = random.sample(image_files, num_samples)

fig, axes = plt.subplots(3, 3, figsize=(11, 11))

for i in range(num_samples):
    image = rand_images[i]
    ax = axes[i // 3, i % 3]
    ax.imshow(plt.imread(os.path.join(Image_dir, image)))
    ax.set_title(f'Image {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from contextlib import redirect_stdout
import logging
from google.colab.patches import cv2_imshow

# Define paths
model_path = '/content/best.pt'  # Path to your trained model
input_folder = '/content/validate/images'           # Path to the folder containing images
label_folder = '/content/validate/labels'           # Path to the folder containing ground truth labels
output_folder = '/content/output'          # Path to save the output images
csv_output_path = '/content/result/1st_model_evaluation_results.csv'  # Path to save the evaluation results CSV
txt_output_path = '/content/result/1st_model_final_conclusion.txt'    # Path to save the final conclusion TXT

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the YOLOv8 model
model = YOLO(model_path)

# Initialize lists to store evaluation results
results_data = []
image_with_detections_count = 0  # Counter for images with detections

# Function to parse label file into a list of ground truth classes
def parse_label_file(label_path):
    classes = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = parts[0]
            # Map class IDs to class names if necessary
            class_name = {
                '5': '50kmh',
                '6': '60kmh',
                '7': '70kmh',
                '8': '80kmh',
                '9': '90kmh'
            }.get(class_id, class_id)  # Default to class_id if not found
            classes.append(class_name)
    return classes

# Suppress logging messages
logging.getLogger().setLevel(logging.ERROR)

# Suppress detailed output from YOLO model predictions
with open(os.devnull, 'w') as fnull:
    with redirect_stdout(fnull):  # Suppress unwanted output
        for image_name in os.listdir(input_folder):
            # Full path to the image
            image_path = os.path.join(input_folder, image_name)

            # Check if the file is an image
            if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                continue

            # Perform detection
            results = model.predict(source=image_path, verbose=False)

            # Extract detected class names
            detected_classes = [model.names[int(cls)] for cls in results[0].boxes.cls]

            # If detections exist, increment the counter
            if detected_classes:
                image_with_detections_count += 1

            # Corresponding label file
            label_file = image_name.replace('.jpg', '.txt')  # Adjust extension if necessary
            label_path = os.path.join(label_folder, label_file)

            # Parse ground truth classes from label file
            if os.path.exists(label_path):
                gt_classes = parse_label_file(label_path)
            else:
                gt_classes = []

            # Compare detections with ground truth classes
            tp = len(set(detected_classes) & set(gt_classes))  # True Positives: correct class detections
            fp = len(detected_classes) - tp  # False Positives: incorrect class detections
            fn = len(gt_classes) - tp  # False Negatives: missed ground truth classes

            # Save results
            results_data.append({
                'Image': image_name,
                'True Positives': tp,
                'False Positives': fp,
                'False Negatives': fn,
                'Detected Classes': detected_classes,
                'Ground Truth Classes': gt_classes
            })

            # Save the output image with detections
            detected_img = results[0].plot()
            output_path = os.path.join(output_folder, f"detected_{image_name}")
            cv2.imwrite(output_path, detected_img)
            # Uncomment the line below if you want to display the image in Colab
            cv2_imshow(detected_img)

# Convert results to a DataFrame and save to CSV
df = pd.DataFrame(results_data)
df['Precision'] = df['True Positives'] / (df['True Positives'] + df['False Positives'])
df['Recall'] = df['True Positives'] / (df['True Positives'] + df['False Negatives'])
df['F1 Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

# Save evaluation results to CSV
df.to_csv(csv_output_path, index=False)

# Calculate overall evaluation metrics
total_tp = df['True Positives'].sum()
total_fp = df['False Positives'].sum()
total_fn = df['False Negatives'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Save final conclusion to TXT file
with open(txt_output_path, 'w') as f:
    f.write(f"Overall Evaluation Metrics:\n")
    f.write(f"Total True Positives: {total_tp}\n")
    f.write(f"Total False Positives: {total_fp}\n")
    f.write(f"Total False Negatives: {total_fn}\n")
    f.write(f"Overall Precision: {overall_precision:.2f}\n")
    f.write(f"Overall Recall: {overall_recall:.2f}\n")
    f.write(f"Overall F1 Score: {overall_f1:.2f}\n")

print("Evaluation completed and results saved to:", csv_output_path)
print("Final conclusion saved to:", txt_output_path)

# Print the number of images with detections
print(f"Total number of images with detections: {image_with_detections_count}")


In [17]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from contextlib import redirect_stdout
import logging
from google.colab.patches import cv2_imshow

# Define paths
model_path = '/content/best.pt'  # Path to your trained model
input_folder = '/content/validate/images'           # Path to the folder containing images
label_folder = '/content/validate/labels'           # Path to the folder containing ground truth labels
output_folder = '/content/output'          # Path to save the output images
csv_output_path = '/content/result/1st_model_evaluation_results.csv'  # Path to save the evaluation results CSV
txt_output_path = '/content/result/1st_model_final_conclusion.txt'    # Path to save the final conclusion TXT

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the YOLOv8 model
model = YOLO(model_path)

# Initialize lists to store evaluation results
results_data = []
image_with_detections_count = 0  # Counter for images with detections

# Function to parse label file into a list of ground truth classes
def parse_label_file(label_path):
    classes = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = parts[0]
            # Map class IDs to class names if necessary
            class_name = {
                '5': '50kmh',
                '6': '60kmh',
                '7': '70kmh',
                '8': '80kmh',
                '9': '90kmh'
            }.get(class_id, class_id)  # Default to class_id if not found
            classes.append(class_name)
    return classes

# Suppress logging messages
logging.getLogger().setLevel(logging.ERROR)

# Suppress detailed output from YOLO model predictions
with open(os.devnull, 'w') as fnull:
    with redirect_stdout(fnull):  # Suppress unwanted output
        for image_name in os.listdir(input_folder):
            # Full path to the image
            image_path = os.path.join(input_folder, image_name)

            # Check if the file is an image
            if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                continue

            # Perform detection
            results = model.predict(source=image_path, verbose=False)

            # Extract detected class names
            detected_classes = [model.names[int(cls)] for cls in results[0].boxes.cls]

            # If detections exist, increment the counter
            if detected_classes:
                image_with_detections_count += 1

            # Corresponding label file
            label_file = image_name.replace('.jpg', '.txt')  # Adjust extension if necessary
            label_path = os.path.join(label_folder, label_file)

            # Parse ground truth classes from label file
            if os.path.exists(label_path):
                gt_classes = parse_label_file(label_path)
            else:
                gt_classes = []

            # Compare detections with ground truth classes
            tp = len(set(detected_classes) & set(gt_classes))  # True Positives: correct class detections
            fp = len(detected_classes) - tp  # False Positives: incorrect class detections
            fn = len(gt_classes) - tp  # False Negatives: missed ground truth classes

            # Save results
            results_data.append({
                'Image': image_name,
                'True Positives': tp,
                'False Positives': fp,
                'False Negatives': fn,
                'Detected Classes': detected_classes,
                'Ground Truth Classes': gt_classes
            })

            # Save the output image with detections
            detected_img = results[0].plot()
            output_path = os.path.join(output_folder, f"detected_{image_name}")
            cv2.imwrite(output_path, detected_img)
            # Uncomment the line below if you want to display the image in Colab
            #cv2_imshow(detected_img)

# Convert results to a DataFrame and save to CSV
df = pd.DataFrame(results_data)
df['Precision'] = df['True Positives'] / (df['True Positives'] + df['False Positives'])
df['Recall'] = df['True Positives'] / (df['True Positives'] + df['False Negatives'])
df['F1 Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

# Save evaluation results to CSV
df.to_csv(csv_output_path, index=False)

# Calculate overall evaluation metrics
total_tp = df['True Positives'].sum()
total_fp = df['False Positives'].sum()
total_fn = df['False Negatives'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Save final conclusion to TXT file
with open(txt_output_path, 'w') as f:
    f.write(f"Overall Evaluation Metrics:\n")
    f.write(f"Total True Positives: {total_tp}\n")
    f.write(f"Total False Positives: {total_fp}\n")
    f.write(f"Total False Negatives: {total_fn}\n")
    f.write(f"Overall Precision: {overall_precision:.2f}\n")
    f.write(f"Overall Recall: {overall_recall:.2f}\n")
    f.write(f"Overall F1 Score: {overall_f1:.2f}\n")

print("Evaluation completed and results saved to:", csv_output_path)
print("Final conclusion saved to:", txt_output_path)

# Print the number of images with detections
print(f"Total number of images with detections: {image_with_detections_count}")


Evaluation completed and results saved to: /content/result/1st_model_evaluation_results.csv
Final conclusion saved to: /content/result/1st_model_final_conclusion.txt
Total number of images with detections: 393


# Validate the team_mode with validate_image_data

In [ ]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from contextlib import redirect_stdout
import logging

# Define paths
model_path = '/content/finish_best_0827.pt'  # Path to your trained model
input_folder = '/content/validate/images'           # Path to the folder containing images
label_folder = '/content/validate/labels'           # Path to the folder containing ground truth labels
output_folder = '/content/output'          # Path to save the output images
csv_output_path = '/content/result/team_model_evaluation_results.csv'  # Path to save the evaluation results CSV
txt_output_path = '/content/result/final_conclusion.txt'    # Path to save the final conclusion TXT

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the YOLOv8 model
model = YOLO(model_path)

# Initialize lists to store evaluation results
results_data = []
image_with_detections_count = 0  # Counter for images with detections

# Function to parse label file into a list of ground truth classes
def parse_label_file(label_path):
    classes = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = parts[0]
            # Map class IDs to class names if necessary
            class_name = {
                '5': 'SpeedLimit_50',
                '6': 'SpeedLimit_60',
                '7': 'SpeedLimit_70',
                '8': 'SpeedLimit_80',
                '9': 'SpeedLimit_90'
            }.get(class_id, class_id)  # Default to class_id if not found
            classes.append(class_name)
    return classes

# Suppress logging messages
logging.getLogger().setLevel(logging.ERROR)

# Suppress detailed output from YOLO model predictions
with open(os.devnull, 'w') as fnull:
    with redirect_stdout(fnull):  # Suppress unwanted output
        for image_name in os.listdir(input_folder):
            # Full path to the image
            image_path = os.path.join(input_folder, image_name)

            # Check if the file is an image
            if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                continue

            # Perform detection
            results = model.predict(source=image_path, verbose=False)

            # Extract detected class names
            detected_classes = [model.names[int(cls)] for cls in results[0].boxes.cls]

            # If detections exist, increment the counter
            if detected_classes:
                image_with_detections_count += 1

            # Corresponding label file
            label_file = image_name.replace('.jpg', '.txt')  # Adjust extension if necessary
            label_path = os.path.join(label_folder, label_file)

            # Parse ground truth classes from label file
            if os.path.exists(label_path):
                gt_classes = parse_label_file(label_path)
            else:
                gt_classes = []

            # Compare detections with ground truth classes
            tp = len(set(detected_classes) & set(gt_classes))  # True Positives: correct class detections
            fp = len(detected_classes) - tp  # False Positives: incorrect class detections
            fn = len(gt_classes) - tp  # False Negatives: missed ground truth classes

            # Save results
            results_data.append({
                'Image': image_name,
                'True Positives': tp,
                'False Positives': fp,
                'False Negatives': fn,
                'Detected Classes': detected_classes,
                'Ground Truth Classes': gt_classes
            })

            # Save the output image with detections
            detected_img = results[0].plot()
            output_path = os.path.join(output_folder, f"detected_{image_name}")
            cv2.imwrite(output_path, detected_img)
            # Uncomment the line below if you want to display the image in Colab
            cv2_imshow(detected_img)

# Convert results to a DataFrame and save to CSV
df = pd.DataFrame(results_data)
df['Precision'] = df['True Positives'] / (df['True Positives'] + df['False Positives'])
df['Recall'] = df['True Positives'] / (df['True Positives'] + df['False Negatives'])
df['F1 Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

# Save evaluation results to CSV
df.to_csv(csv_output_path, index=False)

# Calculate overall evaluation metrics
total_tp = df['True Positives'].sum()
total_fp = df['False Positives'].sum()
total_fn = df['False Negatives'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Save final conclusion to TXT file
with open(txt_output_path, 'w') as f:
    f.write(f"Overall Evaluation Metrics:\n")
    f.write(f"Total True Positives: {total_tp}\n")
    f.write(f"Total False Positives: {total_fp}\n")
    f.write(f"Total False Negatives: {total_fn}\n")
    f.write(f"Overall Precision: {overall_precision:.2f}\n")
    f.write(f"Overall Recall: {overall_recall:.2f}\n")
    f.write(f"Overall F1 Score: {overall_f1:.2f}\n")

print("Evaluation completed and results saved to:", csv_output_path)
print("Final conclusion saved to:", txt_output_path)

# Print the number of images with detections
print(f"Total number of images with detections: {image_with_detections_count}")


In [7]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from contextlib import redirect_stdout
import logging

# Define paths
model_path = '/content/finish_best_0827.pt'  # Path to your trained model
input_folder = '/content/validate/images'           # Path to the folder containing images
label_folder = '/content/validate/labels'           # Path to the folder containing ground truth labels
output_folder = '/content/output'          # Path to save the output images
csv_output_path = '/content/result/team_model_evaluation_results.csv'  # Path to save the evaluation results CSV
txt_output_path = '/content/result/final_conclusion.txt'    # Path to save the final conclusion TXT

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the YOLOv8 model
model = YOLO(model_path)

# Initialize lists to store evaluation results
results_data = []
image_with_detections_count = 0  # Counter for images with detections

# Function to parse label file into a list of ground truth classes
def parse_label_file(label_path):
    classes = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = parts[0]
            # Map class IDs to class names if necessary
            class_name = {
                '5': 'SpeedLimit_50',
                '6': 'SpeedLimit_60',
                '7': 'SpeedLimit_70',
                '8': 'SpeedLimit_80',
                '9': 'SpeedLimit_90'
            }.get(class_id, class_id)  # Default to class_id if not found
            classes.append(class_name)
    return classes

# Suppress logging messages
logging.getLogger().setLevel(logging.ERROR)

# Suppress detailed output from YOLO model predictions
with open(os.devnull, 'w') as fnull:
    with redirect_stdout(fnull):  # Suppress unwanted output
        for image_name in os.listdir(input_folder):
            # Full path to the image
            image_path = os.path.join(input_folder, image_name)

            # Check if the file is an image
            if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                continue

            # Perform detection
            results = model.predict(source=image_path, verbose=False)

            # Extract detected class names
            detected_classes = [model.names[int(cls)] for cls in results[0].boxes.cls]

            # If detections exist, increment the counter
            if detected_classes:
                image_with_detections_count += 1

            # Corresponding label file
            label_file = image_name.replace('.jpg', '.txt')  # Adjust extension if necessary
            label_path = os.path.join(label_folder, label_file)

            # Parse ground truth classes from label file
            if os.path.exists(label_path):
                gt_classes = parse_label_file(label_path)
            else:
                gt_classes = []

            # Compare detections with ground truth classes
            tp = len(set(detected_classes) & set(gt_classes))  # True Positives: correct class detections
            fp = len(detected_classes) - tp  # False Positives: incorrect class detections
            fn = len(gt_classes) - tp  # False Negatives: missed ground truth classes

            # Save results
            results_data.append({
                'Image': image_name,
                'True Positives': tp,
                'False Positives': fp,
                'False Negatives': fn,
                'Detected Classes': detected_classes,
                'Ground Truth Classes': gt_classes
            })

            # Save the output image with detections
            detected_img = results[0].plot()
            output_path = os.path.join(output_folder, f"detected_{image_name}")
            cv2.imwrite(output_path, detected_img)
            # Uncomment the line below if you want to display the image in Colab
            # cv2_imshow(detected_img)

# Convert results to a DataFrame and save to CSV
df = pd.DataFrame(results_data)
df['Precision'] = df['True Positives'] / (df['True Positives'] + df['False Positives'])
df['Recall'] = df['True Positives'] / (df['True Positives'] + df['False Negatives'])
df['F1 Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

# Save evaluation results to CSV
df.to_csv(csv_output_path, index=False)

# Calculate overall evaluation metrics
total_tp = df['True Positives'].sum()
total_fp = df['False Positives'].sum()
total_fn = df['False Negatives'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Save final conclusion to TXT file
with open(txt_output_path, 'w') as f:
    f.write(f"Overall Evaluation Metrics:\n")
    f.write(f"Total True Positives: {total_tp}\n")
    f.write(f"Total False Positives: {total_fp}\n")
    f.write(f"Total False Negatives: {total_fn}\n")
    f.write(f"Overall Precision: {overall_precision:.2f}\n")
    f.write(f"Overall Recall: {overall_recall:.2f}\n")
    f.write(f"Overall F1 Score: {overall_f1:.2f}\n")

print("Evaluation completed and results saved to:", csv_output_path)
print("Final conclusion saved to:", txt_output_path)

# Print the number of images with detections
print(f"Total number of images with detections: {image_with_detections_count}")


ImportError: cannot import name 'YOLO' from 'ultralytics' (unknown location)

In [3]:
rm -r /content/ultralytics